In [1]:
import numpy as np
import pandas as pd

import time

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [16]:
def plotArray(x, y, text='(x,y)', title='', xlabel='', ylabel='', stem=False):
  fig = make_subplots(rows=1, cols=1, shared_xaxes=True)

  if(stem):
    fig.add_trace(
        go.Scatter(x=x, y=y, text=text, name='analog', mode='markers', marker=dict(size=5, line=dict(width=1))),
        row=1, col=1
    )
  else:
    fig.add_trace(
        go.Scatter(x=x, y=y, text=text, name='analog'),
        row=1, col=1
    )
  fig.update_layout(height=320, width=640, title_text=title)
  fig.update_yaxes(title_text=ylabel, showgrid=True)
  fig.update_xaxes(title_text=xlabel, showgrid=True)
  fig.show()

In [17]:
wd = 'C:/Users/levyg/Documents/PlatformIO/Projects/tcc_project/mems2sd_esp32/data/'
df = pd.read_csv(wd + 'data_example.txt', 
                sep=',', 
                decimal='.')
df.head()

,data,time
0,0,0.0
1,1,0.1
2,2,0.2
3,3,0.3
4,4,0.4


In [18]:
plotArray(df['time'], df['data'], text='(t,s)', title='Amplitude of signal in time', xlabel='Time (s)', ylabel='Amplitude')